In [2]:
import numpy as np
import tensorflow as tf

# %matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.colors as colors 

import ipysh

%aimport models.base

import Hunch_utils  as Htls
import Hunch_lsplot as Hplt
import Hunch_tSNEplot as Hsne

%aimport Dataset_QSH

%aimport models.AEFIT4
%aimport models.AEFIT5
%aimport models.Compose

# ipysh.Bootstrap_support.debug()

In [ ]:
#
# TEST QSH
#

In [3]:
qsh = Dataset_QSH.Dataset_QSH()
import os
file = ipysh.abs_builddir+'/te_db_r15.npy'
if os.path.isfile(file):
    qsh.load(file)
else:
    qsh.load(ipysh.abs_builddir+'/te_db_2.npy')
    qsh.rebalance_prel(15)
    qsh.save(ipysh.abs_builddir+'/te_db_r15.npy')
    
qsh.shuffle()
qsh.clean_up_poorcurves(5)
qsh.dim = 15
qsh.set_null(np.nan)
qsh.set_normal_positive(['prel','te','tbordo','tcentro'])
# qsh.unbias_mean(0.5, 'te')
# qsh.set_normal_positive()
# qsh.clip_values(0.1,0.6)

print("QSH rebalanced 15 points size: ", len(qsh))

QSH rebalanced 15 points size:  61519


In [ ]:
qsh['te'][0]

In [4]:
def _map(x,y,t):
    xy = tf.concat([x,y], axis=0)
    t = tf.random.normal([2],mean=0.5, stddev=0.2)
    t = tf.clip_by_value(t, clip_value_min=0., clip_value_max=1.)
    return (xy,t),(xy,t)    
ds = qsh.tf_tuple_compose(['prel','te','tbordo_tcentro']).map(lambda x,y,t: ((tf.concat([x,y], axis=0),t),(tf.concat([x,y], axis=0),t)) )
#ds = qsh.tf_tuple_compose(['prel','te','tcentro_tcentro']).map(lambda x,y,t: (_map(x,y,t)) )
# [x for x in ds.batch(2).take(1)][0]



W0825 17:04:30.139169 140063349368640 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/data/ops/dataset_ops.py:504: py_func (from tensorflow.python.ops.script_ops) is deprecated and will be removed in a future version.
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    


In [ ]:
m = models.AEFIT5.AEFIT5(latent_dim=2, feature_dim=30,  dprate=0., scale=1, beta=0., name='sxr_not_composed')

In [ ]:
m.fit(ds.map(lambda x,y: (x[0],x[0]) ).batch(100, drop_remainder=True), epochs=3)

In [ ]:
qsh_pos = 0

In [ ]:
fig = plt.figure('test_curve',figsize=(18, 6))
plt.clf()
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)    
# ax1.set_xlim(-2.,2.)
ax2.set_ylim(0.,1.)


for xy in qsh.ds_array.skip(qsh_pos).batch(1).take(1):    
    xy,_ = xy
    x,y = tf.split(xy,2, axis=1)
    ax2.scatter(x,y,s=80)
    me,_  = m.encode(xy, training=False)
    gpt = me[0].numpy()
    ax1.scatter(gpt[0],gpt[1])
    
    XY = m.decode(me,apply_sigmoid=True, training=False)
    X,Y = tf.split(XY[0], 2)
    X,Y = (X.numpy(), Y.numpy())
    ax2.scatter(X,Y,s=40)

print(qsh_pos)    
qsh_pos += 1

In [ ]:
p = Hplt.LSPlotBokeh()
p.set_model(m)
p.set_data(qsh, counts=1000)
p.plot(notebook_url='http://172.17.0.2:8888')

In [8]:
m1 = models.AEFIT5.AEFIT5(latent_dim=30, feature_dim=30,  dprate=0., scale=1, beta=0., name='sxr', geometry=[])
m2 = models.AEFIT5.AEFIT5(latent_dim=2, feature_dim=2,  dprate=0., scale=1, beta=0., name='temp', geometry=[])
hm_feature_dim = m1.latent_dim + m2.latent_dim
hm = models.AEFIT5.AEFIT5(latent_dim=2, feature_dim=hm_feature_dim, beta=0., scale=1, name='hidden', geometry=[20,20,10])

AEFIT5 ready:
AEFIT5 ready:
AEFIT5 ready:


In [9]:
h = models.Compose.Compose().set_model(hm).compose([m1,m2])
m1.trainable = True
m2.trainable = True
hm.trainable = True
h.compile()

[(None, 30), (None, 2)]


In [10]:
h.fit(ds.batch(100), epochs=6)

Epoch 1/6


ValueError: Variable <tf.Variable 'na_n_dense_5/kernel:0' shape=(32, 32) dtype=float32> has `None` for gradient. Please make sure that all of your ops have a gradient defined (i.e. are differentiable). Common ops without gradient: K.argmax, K.round, K.eval.

In [ ]:
# [ v.name for v in h2.trainable_variables ]
def plot(xy):
    plt.figure('data')
    ax = plt.gca()
    ax.set_ylim(0.,1.)
    x,y = tf.split(xy[0], num_or_size_splits=2)
    plt.plot(x,y,'.')
d = [x for x in ds.shuffle(1000).batch(1).take(100)][0]
d0 = d[0][0]
y0 = h(d[0], training=False)
y0 = tf.sigmoid(y0[0][0])
plot(d0)
plot(y0)



In [ ]:
fig = plt.figure('test_curve',figsize=(18, 6))
plt.clf()
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)    
# ax1.set_xlim(-2.,2.)
ax2.set_ylim(0.,1.)

sx = []
sy = []
for xy in ds.batch(1).take(1000):
    xy,_ = xy
    x,y = tf.split(xy[0],2, axis=1)
    me,_  = h.encode(xy, training=False)
    gpt = me[0].numpy()
    #ax1.scatter(gpt[0],gpt[1])
    sx.append(gpt[0])
    sy.append(gpt[1])

ax1.scatter(sx,sy)
    
for xy in ds.shuffle(100).batch(1).take(1):    
    xy,_ = xy
    x,y = tf.split(xy[0],2, axis=1)
    ax2.scatter(x,y,s=80)
    me,_  = h.encode(xy, training=False)
    gpt = me[0].numpy()
    ax1.scatter(gpt[0],gpt[1])
    
    XY = h.decode(me, training=False)[0][0]
    XY = tf.sigmoid(XY)
    X,Y = tf.split(XY[0], num_or_size_splits=2)
    X,Y = (X.numpy(), Y.numpy())
    ax2.scatter(X,Y,s=40)

print(qsh_pos)    
qsh_pos += 1

In [ ]:
# p = Hplt.LSPlotBokeh()
# p.set_model(h)
# p.set_data(, counts=1000)
# p.plot(notebook_url='http://172.17.0.2:8888')